#Chapter 06
###파이썬 고급 주제

###6.1 멀티 프로세스와 멀티 스레드
*'프로세스'* : 운영 체제에서 실행되는 각 프로그램
#####각 프로세스에는 하나 이상의 *'스레드'* 가 있다

*   ***프로세스***: 운영체제로부터 자원을 할당받은 '작업'의 단위.

*   ***스레드***: 프로세스가 할당받은 자원을 이용하는 '실행 흐름'의 단위.

#####프로그램은 코드 덩어리 파일, 그 프로그램을 실행한 게 프로세스.
비유를 하면  코드(프로세스) , 함수(스레드)




In [ ]:
# subprocess 모듈: 다른 언어로 만들어진 프로그램을 통합, 제어할 수 있게 만드는 모듈
import subprocess
subprocess.run(["echo", "이것은 subprocess입니다."])

# subprocess.CompletedProcess : 완료된 프로세스를 나타내는, run()의 반환 값.

In [ ]:
subprocess.run(["sleep", '10']) # 10초 동안 sleep

In [ ]:
# threading 모듈: 멀티 스레드를 사용    http://pythonstudy.xyz/python/article/24-%EC%93%B0%EB%A0%88%EB%93%9C-Thread
                                      # https://docs.python.org/ko/3.8/library/threading.html
import queue
import threading

q = queue.Queue() # 선입선출(FIFO, First-In, First-Out)큐 객체를 생성

def worker(num):
    while True:
        item = q.get() # 요소 하나를 반환하고 제거
        if item is None: 
            break      # None이면 멈춤
        #작업을 처리한다
        print("스레드 {0} : 처리 완료 {1}".format(num+1, item))
        q.task_done() # 앞서 큐에 넣은 작업이 완료되었음을 나타냄 (작업이 완료되었음을 큐에 알려줌)

if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,)) # target에 있는 함수실행, args를 인자로 받음 (인자가 1개면 ,를 붙여준다)
        t.start() # 스레드를 실행
        threads.append(t)

    for item in range(20):
        q.put(item) # q객체에 item 입력

    #모든 작업이 끝날 때까지 대기한다 (block)
    q.join()
    
    #워커 스레드를 종료한다 (stop)
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

# https://velog.io/@kho5420/Python-Thread-and-Lock-%EC%93%B0%EB%A0%88%EB%93%9C%EC%99%80-%EB%9D%BD

# 락 : 각 스레드의 연산들의 충돌을 방지하기 위함 ('Lock'을 이용해서 쓰레드를 동기화)
#  lock을 acquire하면 해당 쓰레드만 공유 데이터에 접근할 수 있고, lock을 release 해야만 다른 쓰레드에서 공유 데이터에 접근할 수 있습니다

*   자원을 액세스하기 직전에 lock.acquire()를 호출합니다.
*   해당 자원을 사용합니다.
*   처리를 마치면 lock.release()를 호출하여 다른 스레드가 사용할 수 있도록 합니다.



In [ ]:
# 뮤텍스와 세마포어
from threading import Thread, Lock
import threading

def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print("스레드 {0}: {1}\n".format(threading.get_ident(), data))
    finally:
        if thread_safe:
            mutex.release()


if __name__ == "__main__":
    threads = []
    thread_safe = True    # 뮤텍스 사용시 : True, 아니면 False
    mutex = Lock()
    for i in range(20):
        t = Thread(target=worker, args=(mutex, i, thread_safe))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

In [ ]:
# https://qkqhxla1.tistory.com/15
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()

    def acquire(self, name):
        with self.lock:
            self.active.append(name)
            print("획득: {0} | 스레드 풀: {1}".format(name, self.active))

    def release(self, name):
        with self.lock:
            self.active.remove(name)
            print("반환: {0} | 스레드 풀: {1}".format(name, self.active))

# with: 자원을 획득하고 사용 후 반납해야 하는 경우 주로 사용

def worker(semaphore, pool):   # worker 함수 : 이름 지정 -> acquire() -> 1초간 sleep -> release()
    with semaphore:
        name = threading.currentThread().getName() # 현재 스레드의 이름 지정
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)

if __name__ == '__main__':
    threads = []
    pool = ThreadPool()  # 객체 생성
    semaphore = threading.Semaphore(3) # 세마포어 객체 생성, 한번에 실행될 스레드 3개로 제한
    for i in range(10):
        t = threading.Thread(target=worker, name="스레드 " + str(i), args=(semaphore, pool))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

In [ ]:
import threading
import time

sem = threading.Semaphore(3)

class RestrictedArea(threading.Thread):
    def run(self):
        for x in range(3):
            msg = 'Threading Semaphore TEST : %s' % self.getName()
            sem.acquire()

            print(msg)  #  <-- 여기가 3개의 스레드만이 존재할 수 있는 영역
            time.sleep(2)
            sem.release()

threads = []

# 생성할 스레드의 개수, 여기서는 10개의 스레드 생성
for i in range(10):
    threads.append(RestrictedArea())
for th in threads:
    th.start()
for th in threads:
    th.join()
print('Finish All Threading')

**데드락** (교착상태) :두 개 이상의 프로세스 (or 스레드) 가 서로 상대방의 작업이 끝나기만을 기다려서 결과적으로 아무것도 완료되지 못하는 상태
#####해결방안 : 프로그램에서 락을 할당하고, 락을 순서대로 획득하면 된다.
**스핀락** : 바쁜 대기 (busy waiting) 의 형태 , 진입이 불가능할 때, 진입이 가능할 때까지 반복문을 돌면서 재시도하는 방식으로 구현된 락

In [ ]:
# 스레딩에 대한 구글 파이썬 스타일 가이드
# 원자성: 여러개의 쓰레드가 있을 때 특정 시점에 어떤 메소드를 두개 이상의 쓰레드가 동시에 호출 못한다는 것
# 어떠한 작업이 실행될때 언제나 완전하게 진행되어 종료되거나, 그럴 수 없는 경우 실행을 하지 않는 경우를 말한다.
# 원자성을 가지는 작업은 실행되어 진행되다가 종료하지 않고 중간에서 멈추는 경우는 있을 수 없다.
import threading
import time

def consumer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 대기".format(name))
        cond.wait() # 통지되거나 시간제한이 만료될 때까지 기다림
        print("{0} 자원 소비".format(name))

def producer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()

if __name__ == "__main__":
    condition = threading.Condition()
    consumer1 = threading.Thread(name="소비자1", target=consumer, args=(condition,))
    consumer2 = threading.Thread(name="소비자2", target=consumer, args=(condition,)) # 시작, 대기까지 -> wait()
    producer = threading.Thread(name="생산자", target=producer, args=(condition,))   # 소비자에게 알림 -> notifyAll() -> 소비자 자원소비

    consumer1.start()
    consumer2.start()
    producer.start()

#   https://velog.io/@wltjs10645/Python-thread2